# Eat Safe, Love

## Notebook Set Up

In [5]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [7]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [9]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [12]:
# review the collections in our database
collections = db.list_collection_names()

# Print the collections
print("Collections in 'uk_food' database:")
for collection in collections:
    print(collection)

Collections in 'uk_food' database:
establishments


In [14]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [23]:
# Find the establishments with a hygiene score of 20
query = query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of documents with a hygiene score of 20: {count}")

# Display the first document in the results using pprint
first_doc = establishments.find_one(query)
pprint(first_doc)

Number of documents with a hygiene score of 20: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': None,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('669ae3f5f61af7ec2c2ee85e'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}]

In [25]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)
df = pd.DataFrame(list(results))
# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")
# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in DataFrame: 41
                        _id   FHRSID  ChangesByServerID  \
0  669ae3f5f61af7ec2c2ee85e   110681                  0   
1  669ae3f5f61af7ec2c2eebe6   612039                  0   
2  669ae3f6f61af7ec2c2eeee7   730933                  0   
3  669ae3f6f61af7ec2c2ef0d2   172735                  0   
4  669ae3f6f61af7ec2c2ef0e2   172953                  0   
5  669ae3f6f61af7ec2c2efa7f   512854                  0   
6  669ae3f6f61af7ec2c2efca4  1537089                  0   
7  669ae3f6f61af7ec2c2f11ce   155648                  0   
8  669ae3f6f61af7ec2c2f1615  1012883                  0   
9  669ae3f6f61af7ec2c2f1e26   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532       

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [45]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': 'London', 'RatingValue': {'$gte': 4}}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of documents with a RatingValue >= 4 in London: {count}")
# Display the first document in the results using pprint
first_doc = establishments.find_one(query)
pprint(first_doc)


Number of documents with a RatingValue >= 4 in London: 0
None


In [47]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)
df = pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")
# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [61]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.5074
longitude = -0.1278

query = query = {
    'RatingValue': 5,
    'geocode.latitude': {'$gte': latitude - degree_search, '$lte': latitude + degree_search},
    'geocode.longitude': {'$gte': longitude - degree_search, '$lte': longitude + degree_search}
}
sort = [('scores.Hygiene', 1)]
limit = 5
# Print the results
results = establishments.find(query).sort(sort).limit(limit)
for result in results:
    pprint(result)

In [63]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(list(results))
print(df)

Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [68]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {'$match': {'scores.Hygiene': 0}},
    {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}}
]

# Print the number of documents in the result
results = list(establishments.aggregate(pipeline))
print(f"Number of documents in the result: {len(results)}")

# Print the first 10 results
pprint(results[:10])


Number of documents in the result: 56
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [70]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))


Number of rows in DataFrame: 56
               _id  count
0           Thanet   1130
1        Greenwich    882
2        Maidstone    713
3           Newham    711
4            Swale    686
5       Chelmsford    680
6           Medway    672
7           Bexley    607
8  Southend-On-Sea    586
9         Tendring    542
